# Проект 3. О вкусной и здоровой пище  
## Юнит 3. Введение в машинное обучение 
### Skillfactory: DST-10
---
## Какой кейс решаем?  
Представьте, что Вы работаете DS в компании TripAdvisor. Одна из проблем TripAdvisorа — это нечестные рестораны, которые накручивают себе рейтинг. Одним из способов нахождения таких ресторанов, является построение модели, которая предсказывает рейтинг ресторана. Если предсказания модели сильно отличаются от фактического результата, то, возможно, ресторан играет нечестно, и его стоит проверить.
## 2. Разбираемся с данными  
в этом модуле вы будете работать с датасетом, содержащим сведения о 40 000 ресторанах Европы, а модель, которую вы будете обучать, должна будет предсказывать рейтинг ресторана по данным сайта TripAdvisor на основе имеющихся в датасете данных.  

Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:  
- Restaurant_id — идентификационный номер ресторана / сети ресторанов;
- City — город, в котором находится ресторан;
- Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
- Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
- Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
- Price Range — диапазон цен в ресторане;
- Number of Reviews — количество отзывов о ресторане;
- Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
- URL_TA — URL страницы ресторана на TripAdvosor;
- ID_TA — идентификатор ресторана в базе данных TripAdvisor.

In [55]:
# импортируем все необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split # Загружаем специальный инструмент для разбивки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели 
from collections import Counter
import datetime
import os
# import re
# import math
# import copy
print(os.listdir('../rds_3/data'))

['main_task.csv']


In [57]:
# считываем исходный датасет
df = pd.read_csv('../rds_3/data/main_task.csv')

In [58]:
df.head(2)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
Restaurant_id        40000 non-null object
City                 40000 non-null object
Cuisine Style        30717 non-null object
Ranking              40000 non-null float64
Rating               40000 non-null float64
Price Range          26114 non-null object
Number of Reviews    37457 non-null float64
Reviews              40000 non-null object
URL_TA               40000 non-null object
ID_TA                40000 non-null object
dtypes: float64(3), object(7)
memory usage: 2.0+ MB


## 2.1 Задание 1
Какие столбцы НЕ содержат пропущенных (None) значений?  
***Ответ:***
- НЕ СОДЕРЖ - Restaurant_id
- НЕ СОДЕРЖ - City
  - ***СОДЕРЖ - Cuisine Style***
- НЕ СОДЕРЖ - Ranking
- НЕ СОДЕРЖ - Rating
  - ***СОДЕРЖ - Price Range***
  - ***СОДЕРЖ - Number of Reviews***
- НЕ СОДЕРЖ - Reviews
- НЕ СОДЕРЖ - URL_TA
- НЕ СОДЕРЖ - ID_TA

## 2.2 Задание 2
В каких столбцах данные хранятся в числовом формате?  
***Ответ:***
- 
  - ***object - Restaurant_id***
  - ***object - City***
  - ***object - Cuisine Style***
- В ЧИСЛОВОМ - Ranking
- В ЧИСЛОВОМ - Rating
  - ***object - Price Range***
- В ЧИСЛОВОМ - Number of Reviews
  - ***object - Reviews***
  - ***object - URL_TA***
  - ***object - ID_TA***

## 2.3 Задание 3
В каких столбцах хранящиеся данные представляют собой список?

***Ответ:*** - ни в одном из перечисленных. Значения выглядящие как списки, на самом деле, тип object - "строковый" тип


In [60]:
# сделаем из кода построения модели функцию, так как нам придется ее повторять 
def first_model(df):
    # Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
    X = df.drop(['Rating'], axis = 1)  
    y = df['Rating']  
    
    # Наборы данных с меткой "train" будут использоваться для обучения модели, 
    # "test" - для тестирования.  
    # Для тестирования мы будем использовать 25% от исходного датасета.  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

    # Создаём модель  
    regr = RandomForestRegressor(n_estimators=100)  
    
    # Обучаем модель на тестовом наборе данных  
    regr.fit(X_train, y_train)  
    
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
    # Предсказанные значения записываем в переменную y_pred  
    y_pred = regr.predict(X_test) 

    # Сравниваем предсказанные значения (y_pred) с реальными (y_test), 
    # и смотрим насколько они в среднем отличаются  
    # Метрика называется Mean Absolute Error (MAE) и показывает 
    # среднее отклонение предсказанных значений от фактических.  
    print('MAE:', metrics.mean_absolute_error(y_test, y_pred)) 

In [61]:
# запускаем функцию с построением модели и оценкой МАЕ
first_model(df)

ValueError: could not convert string to float: 'id_4702'

## 3.1 Вопрос для размышления  
По какой причине во время обучения модели возникла ошибка Value Error?  

***Ответ:*** - Ошибка возникла при обработке столбца со строковыми значениями. Следовательно, для обучения модели все данные должны быть переданы только в числовом формате. 


In [62]:
# заполняем пропуски 
# сначала заполним числовой ряд средними значениями 
df['Number of Reviews'] = df['Number of Reviews'].fillna(df['Number of Reviews'].mean())

In [63]:
# проверяем заполнение
df['Number of Reviews'].isnull().sum()

0

In [64]:
# теперь запольним Price Range
# сначала заменим все значения на числовые 
dic_value_Price = {'$':1,'$$ - $$$':2,'$$$$':3}
df['Price Range']=df['Price Range'].map(lambda x: dic_value_Price.get(x,x))
# потом заменим на самое популярное
df['Price Range'] = df['Price Range'].fillna(df["Price Range"].value_counts().idxmax())

In [65]:
# проверяем заполнение
df['Price Range'].isnull().sum()

0

In [66]:
# City переводим в числа с помощью метода get_dummies
df_city_dummies = pd.get_dummies(df['City'])
df = pd.concat([df,df_city_dummies], axis=1)

In [67]:
# на этом тренировочном этапе удалим все столбцы, 
# которые сложно заполнить или перевести в числа
df.drop(['Restaurant_id', 'City', 'Cuisine Style', 'Reviews', 'URL_TA', 'ID_TA'], axis=1, inplace=True)

In [68]:
# повторяем построение модели
first_model(df)

MAE: 0.215741


## 4.1 Вопросы о ценах
1. Сколько вариантов непустых значений встречается в столбце Price Range?  
***Ответ:*** 3

2. Как в датафрейме обозначается самый низкий уровень цен?  
***Ответ:*** $

3. Как в датафрейме обозначается самый высокий уровень цен?  
***Ответ:*** $$$$

4. Сколько ресторанов относятся к среднему ценовому сегменту?

In [69]:
# перед решением следующих задач скачиваем заново датасет, 
# потому что мы уже изменили первоначальный датасет
df = pd.read_csv('../rds_3/data/main_task.csv')

temp = len(df[df['Price Range'] == '$$ - $$$'])
task = "4.1"
print(f"Ответ к заданию {task} пункт 4")
print(f"{round(temp,0):=} := ресторанов относятся к среднему ценовому сегменту")

Ответ к заданию 4.1 пункт 4
18412 := ресторанов относятся к среднему ценовому сегменту


## 4.2 Вопрос о городах
Сколько городов представлено в наборе данных?

In [70]:
temp = df['City'].nunique()
task = "4.2"
print(f"Ответ к заданию {task} пункт 4")
print(f"{round(temp,0)} :=городов представлено в наборе")

Ответ к заданию 4.2 пункт 4
31 :=городов представлено в наборе


## 4.3 Вопросы о кухнях
1. Сколько типов кухонь представлено в наборе данных?  
2. Какая кухня представлена в наибольшем количестве ресторанов? Введите название кухни без кавычек или апострофов.  
3. Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. Ответ округлите до одного знака после запятой.

In [71]:
# сначала заполним пропуски в Cuisine Style
# можно было бы какими-нибудь самыми популярными или медианными значениями по городам
# но мы пока на этапе обучения поэтому можно просто поставить какую-нибудь "другую" кухню
df['Cuisine Style'] = df['Cuisine Style'].fillna("['Other']")
df['Cuisine Style'] = df['Cuisine Style'].str.findall(r"'(\b.*?\b)'") 

# создадим список списков кухонь
# temp_list = [i.split('') for i in df['Cuisine Style']]
temp_list = df['Cuisine Style'].tolist()

# но у нас получился список списков, с которым работать неудобно
# напишем функцию распаковки списка списков 
# !!! взял данный подход из rds1, потому что он достаточно быстрый!!!)
def list_unrar(list_of_lists):
    result=[]
    for lst in list_of_lists:
      result.extend(lst)
    return result

#  записываем распакованный список в Counter
temp_counter=Counter(list_unrar(temp_list))

In [72]:
temp1 = len(temp_counter) - 1  # минусуем нашу одну "другую" кухню
task = "4.3"
print(f"Ответ к заданию {task} пункт 1")
print(f"{round(temp1,0)} := типов кухонь представлено в наборе данных")

temp2 = temp_counter.most_common()[0][0]
print(f"Ответ к заданию {task} пункт 2")
print(f"{temp2} := кухня представлена в наибольшем количестве ресторанов")

Ответ к заданию 4.3 пункт 1
125 := типов кухонь представлено в наборе данных
Ответ к заданию 4.3 пункт 2
Vegetarian Friendly := кухня представлена в наибольшем количестве ресторанов


In [73]:
# ну и чтобы не бросать этот критерий Cuisine Style на полдороги, сделаем аналог get_dummies
for cuisine in temp_counter:
    df[cuisine] = df['Cuisine Style'].apply(lambda x: 1 if cuisine in x else 0 ).astype('float64')

In [74]:
# чтобы не удалять столбец и не создавать новый для кол-ва кухонь 
# запишем в текущий кол-во кухонь
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: len(x)).astype('float64')

In [75]:
temp3 = df['Cuisine Style'].mean()
print(f"Ответ к заданию {task} пункт 3")
print(f"среднее количество кухонь предлагается в одном ресторане :={round(temp3,1)}")

Ответ к заданию 4.3 пункт 3
среднее количество кухонь предлагается в одном ресторане :=2.6


## 4.4 Вопросы об отзывах
1. Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.
2. Какое максимальное количество дней отделяет даты публикации отзывов, размещённых на сайте ресторана? Введите количество дней в виде целого числа.

In [76]:
# заполняем пустые значения в Reviews
df['Reviews'] = df['Reviews'].fillna("['no_Reviews']")
# находим даты и записываем в новый столбец в формате даты
df['date_of_Review'] = df['Reviews'].str.findall('\d+/\d+/\d+')


In [77]:
# проверяем
with pd.option_context('display.max_columns', None):
    display(df.head(2))

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,European,French,International,Other,Japanese,Sushi,Asian,Grill,Vegetarian Friendly,Vegan Options,Gluten Free Options,German,Central European,Italian,Pizza,Fast Food,Mediterranean,Spanish,Healthy,Cafe,Thai,Vietnamese,Bar,Pub,Chinese,British,Polish,Fusion,Dutch,Mexican,Venezuelan,South American,Soups,Belgian,Steakhouse,Latin,Barbecue,Argentinean,Irish,Seafood,Swiss,Portuguese,Contemporary,Wine Bar,Greek,Central American,Indian,Middle Eastern,Turkish,Hungarian,Pakistani,Peruvian,Delicatessen,Eastern European,Swedish,Scandinavian,Tibetan,Nepali,Korean,Southwestern,Czech,American,Slovenian,Balti,Street Food,Diner,Brew Pub,Caribbean,Austrian,Moroccan,Halal,Lebanese,Russian,African,Ethiopian,Egyptian,Danish,Brazilian,Ecuadorean,Israeli,Kosher,Gastropub,Australian,Singaporean,Malaysian,Minority Chinese,Scottish,Arabic,Ukrainian,Chilean,Mongolian,Cuban,Persian,Indonesian,Colombian,Jamaican,Norwegian,Hawaiian,Armenian,Taiwanese,Bangladeshi,Sri Lankan,Cambodian,Albanian,New Zealand,Croatian,Central Asian,Filipino,Tunisian,Cajun & Creole,Romanian,Georgian,Polynesian,Azerbaijani,Caucasian,Afghani,Uzbek,Salvadoran,Yunnan,Native American,Canadian,Xinjiang,Burmese,Fujian,Welsh,Latvian,date_of_Review
0,id_5569,Paris,3.0,5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[12/31/2017, 11/20/2017]"
1,id_1535,Stockholm,1.0,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[07/06/2017, 06/19/2016]"


In [78]:
# функция расчета времени от текущего момента до самого раннего отзыва из двух
def time_to_now(row):
    if row['date_of_Review'] == []:
        return None
    return pd.datetime.now() - pd.to_datetime(row['date_of_Review']).max()

# функция расчета времени между отзывами
def time_between_Reviews(row):
    if row['date_of_Review'] == []:
        return None
    return pd.to_datetime(row['date_of_Review']).max() - pd.to_datetime(row['date_of_Review']).min()

# с помощью функций заполняем столбец с кол-вом дней то текущего мента 
# c момента самого раннего отзыва из двух
df['day_to_now'] = df.apply(time_to_now, axis = 1).dt.days

# с помощью функций заполняем столбец с кол-вом дней между отзывами
df['day_between_Reviews'] = df.apply(time_between_Reviews, axis = 1).dt.days

In [83]:
task = '4.4'

temp1 = datetime.date.today() - datetime.timedelta(days=df['day_to_now'].min())
print(f"Ответ к заданию {task} пункт 1")
print(f"самый свежий отзыв был оставлен :={temp1}")

temp2 = df['day_between_Reviews'].max()
print(f"Ответ к заданию {task} пункт 2")
print(f"самый свежий отзыв был оставлен :={temp2}")

Ответ к заданию 4.4 пункт 1
самый свежий отзыв был оставлен :=2018-02-26
Ответ к заданию 4.4 пункт 2
самый свежий отзыв был оставлен :=3207.0


In [80]:
# так как у нас возник вопрос про МАЕ, то сделам необходимые заполнения и удаления столбцов

df['Number of Reviews'] = df['Number of Reviews'].fillna(df['Number of Reviews'].mean())

# теперь заполним Price Range
# сначала заменим все значения на числовые 
dic_value_Price = {'$':1,'$$ - $$$':2,'$$$$':3}
df['Price Range']=df['Price Range'].map(lambda x: dic_value_Price.get(x,x))
# потом заменим на самое популярное
df['Price Range'] = df['Price Range'].fillna(df["Price Range"].value_counts().idxmax())

# City переводим в числа с помощью метода get_dummies
df_city_dummies = pd.get_dummies(df['City']).astype('float64')
df = pd.concat([df,df_city_dummies], axis=1)

# заполняем пустые значения в столбцах с днями 
df['day_to_now'] = df['day_to_now'].fillna(df['day_to_now'].max())
df['day_between_Reviews'] = df['day_between_Reviews'].fillna(0)

# удаляем все не нужное
df.drop(['Restaurant_id', 'City', 'Reviews', 'URL_TA', 'ID_TA', 'date_of_Review' ], axis=1, inplace=True)

In [81]:
# проверяем
with pd.option_context('display.max_columns', None):
    display(df.describe())


,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,European,French,International,Other,Japanese,Sushi,Asian,Grill,Vegetarian Friendly,Vegan Options,Gluten Free Options,German,Central European,Italian,Pizza,Fast Food,Mediterranean,Spanish,Healthy,Cafe,Thai,Vietnamese,Bar,Pub,Chinese,British,Polish,Fusion,Dutch,Mexican,Venezuelan,South American,Soups,Belgian,Steakhouse,Latin,Barbecue,Argentinean,Irish,Seafood,Swiss,Portuguese,Contemporary,Wine Bar,Greek,Central American,Indian,Middle Eastern,Turkish,Hungarian,Pakistani,Peruvian,Delicatessen,Eastern European,Swedish,Scandinavian,Tibetan,Nepali,Korean,Southwestern,Czech,American,Slovenian,Balti,Street Food,Diner,Brew Pub,Caribbean,Austrian,Moroccan,Halal,Lebanese,Russian,African,Ethiopian,Egyptian,Danish,Brazilian,Ecuadorean,Israeli,Kosher,Gastropub,Australian,Singaporean,Malaysian,Minority Chinese,Scottish,Arabic,Ukrainian,Chilean,Mongolian,Cuban,Persian,Indonesian,Colombian,Jamaican,Norwegian,Hawaiian,Armenian,Taiwanese,Bangladeshi,Sri Lankan,Cambodian,Albanian,New Zealand,Croatian,Central Asian,Filipino,Tunisian,Cajun & Creole,Romanian,Georgian,Polynesian,Azerbaijani,Caucasian,Afghani,Uzbek,Salvadoran,Yunnan,Native American,Canadian,Xinjiang,Burmese,Fujian,Welsh,Latvian,day_to_now,day_between_Reviews,Amsterdam,Athens,Barcelona,Berlin,Bratislava,Brussels,Budapest,Copenhagen,Dublin,Edinburgh,Geneva,Hamburg,Helsinki,Krakow,Lisbon,Ljubljana,London,Luxembourg,Lyon,Madrid,Milan,Munich,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.00000,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.0000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.0000,40000.000000,40000.000000,40000.000000,40000.000000,40000.0000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,2.622400,3676.028525,3.993037,1.878600,124.825480,0.251500,0.079750,0.03960,0.232075,0.03660,0.028900,0.075275,0.012250,0.279725,0.112150,0.102825,0.016550,0.034825,0.149100,0.071225,0.042625,0.156925,0.069950,0.015500,0.058125,0.018575,0.012825,0.082425,0.061225,0.028625,0.039875,0.009125,0.014425,0.007350,0.011125,0.001000,0.009300,0.012350,0.0

In [82]:
first_model(df)

MAE: 0.2124
